In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import gkey

In [3]:
home_sales_df = pd.read_csv("Resources/toronto_home_prices_Beza.csv")
home_sales_df.head()

,Index,AreaName,Price ($),lat,lng,Address,City
0,1301,Midtown Toronto,549000,43.672318,-79.388817,#307 - 21 SCOLLARD ST,Toronto
1,1302,Downtown,398900,43.656807,-79.375092,#3604 - 159 DUNDAS ST E,Toronto
2,1303,Niagara,309900,43.637150,-79.400604,#1211 - 15 BRUYERES MEWS,Toronto
3,1304,Old Town,254900,43.653126,-79.369339,#807 - 311 RICHMOND ST E,Toronto
4,1305,Downtown,449900,43.651878,-79.375999,#2505 - 50 LOMBARD ST,Toronto


In [4]:
toronto_home_sales = home_sales_df.drop(columns="Index")
toronto_home_sales["Postal Code"] = ""
toronto_home_sales.head()

,AreaName,Price ($),lat,lng,Address,City,Postal Code
0,Midtown Toronto,549000,43.672318,-79.388817,#307 - 21 SCOLLARD ST,Toronto,
1,Downtown,398900,43.656807,-79.375092,#3604 - 159 DUNDAS ST E,Toronto,
2,Niagara,309900,43.637150,-79.400604,#1211 - 15 BRUYERES MEWS,Toronto,
3,Old Town,254900,43.653126,-79.369339,#807 - 311 RICHMOND ST E,Toronto,
4,Downtown,449900,43.651878,-79.375999,#2505 - 50 LOMBARD ST,Toronto,


In [5]:
# params dictionary to update each iteration
params = {"key": gkey}

# Use the lat/lng we recovered to identify postal code
for index, row in toronto_home_sales.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["latlng"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"

    # make request
    postal_code = requests.get(base_url, params=params)

    # convert to json
    postal_code = postal_code.json()
    
    try:
        toronto_home_sales.loc[index, "Postal Code"] = postal_code["results"][0]["address_components"][7]["long_name"]
    except (IndexError):
        print(f"No Results Found: {lat}, {lng}")

No Results Found: 43.650143, -79.371605
No Results Found: 43.660103, -79.385086
No Results Found: 43.675552, -79.377808
No Results Found: 43.660103, -79.385086
No Results Found: 43.712596999999995, -79.369644
No Results Found: 43.751255, -79.528702
No Results Found: 43.632587, -79.485214
No Results Found: 43.788143, -79.207649
No Results Found: 43.643947999999995, -79.401474
No Results Found: 43.77195, -79.261711
No Results Found: 43.660103, -79.385086
No Results Found: 43.777565, -79.281006
No Results Found: 43.651909, -79.569687
No Results Found: 43.656361, -79.357277
No Results Found: 43.6357, -79.495644
No Results Found: 43.78434, -79.41544300000001
No Results Found: 43.643947999999995, -79.401474
No Results Found: 43.647816, -79.396873
No Results Found: 43.777565, -79.281006
No Results Found: 43.777565, -79.281006
No Results Found: 43.722698, -79.60234799999999
No Results Found: 43.705692, -79.294601
No Results Found: 43.735703, -79.31237800000001
No Results Found: 43.64286, -79.3

In [6]:
# Visualize to confirm airport data appears
toronto_home_sales.head(10)

,AreaName,Price ($),lat,lng,Address,City,Postal Code
0,Midtown Toronto,549000,43.672318,-79.388817,#307 - 21 SCOLLARD ST,Toronto,M5R 1G1
1,Downtown,398900,43.656807,-79.375092,#3604 - 159 DUNDAS ST E,Toronto,M5B 0A9
2,Niagara,309900,43.637150,-79.400604,#1211 - 15 BRUYERES MEWS,Toronto,M5V 0G8
3,Old Town,254900,43.653126,-79.369339,#807 - 311 RICHMOND ST E,Toronto,M5A 1P2
4,Downtown,449900,43.651878,-79.375999,#2505 - 50 LOMBARD ST,Toronto,M5C 2X4
5,Downtown,999999,43.654991,-79.363922,161 PARLIAMENT ST,Toronto,M5A 2Z2
6,Downtown,329900,43.660149,-79.360886,#1304 - 170 SUMACH ST,Toronto,M5A 2B9
7,Toronto,245000,43.640335,-79.424477,#1619 - 38 JOE SHUSTER WAY,Toronto,M6K 0A5
8,Downtown,798000,43.650143,-79.371605,#609 - 39 JARVIS ST,Toronto,
9,Midtown Toronto,549000,43.687836,-79.413376,#506 - 60 MONTCLAIR AVE,Toronto,M5P 1P7


In [7]:
toronto_home_sales.to_csv("output_data/toronto_api_postal_ba.csv")